In [67]:
import pandas as pd
import numpy as np
from vosk import Model, KaldiRecognizer, SetLogLevel
import sys
import os
import wave
import subprocess
import json
from os import listdir
from os.path import isfile, join
import datetime
import time

In [68]:
start = time.time()

files=pd.DataFrame(columns=['folder','path_audio'])

dir_audios="C:/Users/namag/Nestor/DigitalHouse/Desafio/DHDS_TrabajoFinal/Data/Audio"
dir_audios= dir_audios+"/" if dir_audios[-1]!="/" else dir_audios
folders_audio = [dir_audios+f for f in listdir(dir_audios)]

for folders in folders_audio:
    file_audio = [folders+"/"+f for f in listdir(folders)]
    file_audio = pd.DataFrame(file_audio,columns=['path_audio'])
    file_audio['folder'] = folders.split("/")[-1]

    files = files.append(file_audio)

In [69]:
files['file']=files['path_audio'].apply(lambda x: x.split("/")[-1])
aux = pd.Series(files['file'].apply(lambda x: x.replace(".wav","").split("-")[-6:]))
aux = aux.apply(lambda x: [int(a) for a in x])
files['date'] = aux.apply(lambda x: datetime.datetime(x[0], x[1], x[2], hour=x[3], minute=x[4], second=x[5]))


files

,folder,path_audio,file,date
0,C5N,C:/Users/namag/Nestor/DigitalHouse/Desafio/DHD...,2020-08-11-17-59-23.wav,2020-08-11 17:59:23
1,C5N,C:/Users/namag/Nestor/DigitalHouse/Desafio/DHD...,2020-08-14-17-55-49.wav,2020-08-14 17:55:49
0,TN,C:/Users/namag/Nestor/DigitalHouse/Desafio/DHD...,2020-08-11-19-01-34.wav,2020-08-11 19:01:34
1,TN,C:/Users/namag/Nestor/DigitalHouse/Desafio/DHD...,2020-08-14-18-59-25.wav,2020-08-14 18:59:25


In [70]:
folder_model="../Data/Modelos/audio_model_spanish_vosk/model"
model = Model(folder_model)

registros = pd.DataFrame(columns=['path_audio','result','text'])
for file in files['path_audio']:
    wf = wave.open(file, "rb")
    rec = KaldiRecognizer(model, wf.getframerate())    

    while True:
        data = wf.readframes(4000)
        if len(data) == 0:
            break
        if rec.AcceptWaveform(data):
            try:
                new_input = json.loads(rec.Result())
                new_input = pd.DataFrame(new_input)
                new_input['path_audio']=file
                registros = registros.append(new_input)
            except ValueError:
                 continue
        else:
            rec.PartialResult()


In [71]:
registros.reset_index(drop=True,inplace=True)
registros_2 = registros.join(pd.DataFrame(x for x in registros['result']))
registros_2.drop(columns='result',inplace=True)
registros_2.head()

,path_audio,text,conf,end,start,word
0,C:/Users/namag/Nestor/DigitalHouse/Desafio/DHD...,el doctor adolfo rubí stein que había dicho qu...,1.000000,0.90,0.720000,el
1,C:/Users/namag/Nestor/DigitalHouse/Desafio/DHD...,el doctor adolfo rubí stein que había dicho qu...,1.000000,1.23,0.900000,doctor
2,C:/Users/namag/Nestor/DigitalHouse/Desafio/DHD...,el doctor adolfo rubí stein que había dicho qu...,1.000000,1.65,1.230000,adolfo
3,C:/Users/namag/Nestor/DigitalHouse/Desafio/DHD...,el doctor adolfo rubí stein que había dicho qu...,0.952746,2.10,1.680000,rubí
4,C:/Users/namag/Nestor/DigitalHouse/Desafio/DHD...,el doctor adolfo rubí stein que había dicho qu...,1.000000,2.40,2.102699,stein


In [72]:
MinTime = pd.DataFrame(registros_2.groupby(['path_audio','text'])['start'].min())
MaxTime = pd.DataFrame(registros_2.groupby(['path_audio','text'])['end'].max())

registros_3=registros_2[['path_audio','text']]
registros_3 = registros_3.merge(MinTime,on=['path_audio','text'])
registros_3 = registros_3.merge(MaxTime,on=['path_audio','text'])

registros_3.drop_duplicates(inplace=True)
registros_3.reset_index(drop=True,inplace=True)
registros_3.head()

,path_audio,text,start,end
0,C:/Users/namag/Nestor/DigitalHouse/Desafio/DHD...,el doctor adolfo rubí stein que había dicho qu...,0.72,20.16
1,C:/Users/namag/Nestor/DigitalHouse/Desafio/DHD...,fianza de la sociedad pero no sabe cómo salir ...,20.16,38.19
2,C:/Users/namag/Nestor/DigitalHouse/Desafio/DHD...,algunos,39.63,40.35
3,C:/Users/namag/Nestor/DigitalHouse/Desafio/DHD...,funcionarios que realmente hicieron papel muy ...,41.85,61.17
4,C:/Users/namag/Nestor/DigitalHouse/Desafio/DHD...,turismo masivo y eso de jujuy,61.56,64.11


In [73]:
data = files.merge(registros_3, on='path_audio')
print(data.shape)
data.head()

(2122, 7)


,folder,path_audio,file,date,text,start,end
0,C5N,C:/Users/namag/Nestor/DigitalHouse/Desafio/DHD...,2020-08-11-17-59-23.wav,2020-08-11 17:59:23,el doctor adolfo rubí stein que había dicho qu...,0.72,20.16
1,C5N,C:/Users/namag/Nestor/DigitalHouse/Desafio/DHD...,2020-08-11-17-59-23.wav,2020-08-11 17:59:23,fianza de la sociedad pero no sabe cómo salir ...,20.16,38.19
2,C5N,C:/Users/namag/Nestor/DigitalHouse/Desafio/DHD...,2020-08-11-17-59-23.wav,2020-08-11 17:59:23,algunos,39.63,40.35
3,C5N,C:/Users/namag/Nestor/DigitalHouse/Desafio/DHD...,2020-08-11-17-59-23.wav,2020-08-11 17:59:23,funcionarios que realmente hicieron papel muy ...,41.85,61.17
4,C5N,C:/Users/namag/Nestor/DigitalHouse/Desafio/DHD...,2020-08-11-17-59-23.wav,2020-08-11 17:59:23,turismo masivo y eso de jujuy,61.56,64.11


In [74]:
data_text_complete  = pd.DataFrame(columns=['path_audio','text_complete'])
for file in data['path_audio'].unique():
    text = data.loc[data['path_audio'] == file,'text'].unique()
    text_complete = ','.join(text)
    new_input=pd.DataFrame([text_complete], columns=['text_complete'])
    new_input['path_audio'] = file
    data_text_complete = data_text_complete.append(new_input)

data_text_complete.reset_index(drop=True,inplace=True)
data_text_complete=data[['folder','path_audio','file','date']].drop_duplicates().merge(data_text_complete, on = 'path_audio')

In [81]:
pd.set_option('display.max_colwidth',50) ##el segundo parametro indica la cantidad de caracteres a mostrar, con valor -1 muestra todo
data_text_complete

,folder,path_audio,file,date,text_complete
0,C5N,C:/Users/namag/Nestor/DigitalHouse/Desafio/DHD...,2020-08-11-17-59-23.wav,2020-08-11 17:59:23,el doctor adolfo rubí stein que había dicho qu...
1,C5N,C:/Users/namag/Nestor/DigitalHouse/Desafio/DHD...,2020-08-14-17-55-49.wav,2020-08-14 17:55:49,hola muy buenas tardes bienvenidos al diario a...
2,TN,C:/Users/namag/Nestor/DigitalHouse/Desafio/DHD...,2020-08-11-19-01-34.wav,2020-08-11 19:01:34,de ciento cincuenta mil caso corresponder al s...
3,TN,C:/Users/namag/Nestor/DigitalHouse/Desafio/DHD...,2020-08-14-18-59-25.wav,2020-08-14 18:59:25,de los colores que se habían ido arrojó de las...


In [82]:
##Export datasets
data.to_csv("../Data procesada/data_audio.csv")
data_text_complete.to_csv("../Data procesada/data_audio_concatenada.csv")

In [78]:
end = time.time()
print("Tiempo de procesamiento:",round((end - start)/60),"minutos")

Tiempo de procesamiento: 40 minutos
